In [ ]:
from langchain_groq import ChatGroq

import os
from dotenv import load_dotenv
load_dotenv()

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    groq_api_key = os.getenv("API_KEY"),

    # other params...
)

response = llm.invoke("Second person on the moon was ..")
print(response.content)

Edwin "Buzz" Aldrin was the second person to walk on the moon. He followed Neil Armstrong, who was the first person to walk on the moon, during the Apollo 11 mission in 1969.


In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://careers.nike.com/cdn-security-engineer-waf/job/R-48004")
page_data = loader.load().pop().page_content
print(page_data)






















CDN Security Engineer (WAF)










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
        ###SCRAPED DATA FROM WEBSITE
        {page_data}
        ###INSTRUCTION
        The scraped text is from the career's page of a website
        Your job is to extract the job postings and return them in JSON format containing
        following keys: `role`, `experience`, `skills`, and `description`.
        only return the valid JSON.
        ###VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)

{
  "role": "CDN Security Engineer (WAF)",
  "experience": {
    "required": "At least five (5) or more years of relevant experience",
    "desired": "Two (2) or more years of experience providing similar operations and sustainment support with Akamai or other CDN providers"
  },
  "skills": {
    "required": [
      "Strong foundation in Computer Science, software engineering, or a related field",
      "Proven track record of delivering high-quality solutions in a fast-paced, dynamic environment",
      "In-depth knowledge of Akamai's suite of tools and security offerings",
      "Strong understanding of API security requirements, web attack vectors, and threat indicators"
    ],
    "desired": [
      "Professional/functional credentials and certifications in cloud infrastructure, security, or a related field are highly desirable (e.g., AWS Certified Solutions Architect, CISSP, CEH)",
      "Cyber security experience is highly desired, but not required"
    ]
  },
  "description": {

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'CDN Security Engineer (WAF)',
 'experience': {'required': 'At least five (5) or more years of relevant experience',
  'desired': 'Two (2) or more years of experience providing similar operations and sustainment support with Akamai or other CDN providers'},
 'skills': {'required': ['Strong foundation in Computer Science, software engineering, or a related field',
   'Proven track record of delivering high-quality solutions in a fast-paced, dynamic environment',
   "In-depth knowledge of Akamai's suite of tools and security offerings",
   'Strong understanding of API security requirements, web attack vectors, and threat indicators'],
  'desired': ['Professional/functional credentials and certifications in cloud infrastructure, security, or a related field are highly desirable (e.g., AWS Certified Solutions Architect, CISSP, CEH)',
   'Cyber security experience is highly desired, but not required']},
 'description': {'job_summary': "As a CDN Security Engineer (WAF), you'll play 

In [10]:
type(json_res)

dict